In [21]:
from sklearn.datasets import make_blobs
import numpy as np
from keras.models import Sequential, Model
from keras_uncertainty.layers import StochasticDropout, SamplingSoftmax
from keras_uncertainty.models import DisentangledStochasticClassifier, StochasticClassifier
from keras.layers import Dense, Input, Dropout
from keras_uncertainty.utils import numpy_entropy
import matplotlib.pyplot as plt

import sys

sys.path.insert(1, '../')


In [24]:
X, y = make_blobs(n_samples=500, n_features=2, centers=[[-1.5, 1.5],[0, -1.5]], random_state=0)
BATCH_SIZE = 256
NUM_SAMPLES = 100

In [25]:
plt.scatter(X[:,0], X[:, 1], c=y)

In [26]:
from sklearn.utils import shuffle

def partial_shuffle_dataset(X, y, percentage):
    X_noisy, y_noisy = shuffle(X, y)
    np.random.shuffle(y_noisy[:int(len(y_noisy) * percentage)])
    X_noisy, y_noisy = shuffle(X_noisy, y_noisy)
    return X_noisy, y_noisy

X_noisy, y_noisy = partial_shuffle_dataset(X, y, percentage=0.1)
plt.scatter(X_noisy[:,0], X_noisy[:, 1], c=y_noisy)

In [27]:
noises = np.arange(0, 1, 0.1)

In [40]:
from disentanglement.models.architectures import get_blobs_dropout_architecture
from disentanglement.models.entropy_models import mutual_information, expected_entropy, \
    train_entropy_model
from disentanglement.models.multi_head_models import uncertainty, \
    train_disentangle_model
from disentanglement.settings import BATCH_SIZE, NUM_SAMPLES, TEST_MODE
from tqdm import tqdm
from sklearn.metrics import accuracy_score


X_test, y_test = make_blobs(n_samples=200, n_features=2, centers=[[-1.5, 1.5],[0, -1.5]], random_state=1)



disentangling_accuracies = []
disentangling_aleatorics = []
disentangling_epistemics = []

entropy_accuracies = []
entropy_aleatorics = []
entropy_epistemics = []

epochs = 50
architecture_func = get_blobs_dropout_architecture
n_classes = 2

for noise in tqdm(noises):
    X_noisy, y_noisy = partial_shuffle_dataset(X, y, percentage=noise)
    X_test_noisy, y_test_noisy = partial_shuffle_dataset(X_test, y_test, percentage=noise)

    disentangle_model = train_disentangle_model(architecture_func, X_noisy, y_noisy, n_classes,
                                                    epochs=epochs)
    entropy_model = train_entropy_model(architecture_func, X_noisy, y_noisy, n_classes, epochs=epochs)

    pred_mean, pred_ale_std, pred_epi_std = disentangle_model.predict(X_test_noisy, batch_size=BATCH_SIZE)
    entropy_preds = entropy_model.predict_samples(X_test_noisy, num_samples=NUM_SAMPLES, batch_size=BATCH_SIZE)
    disentangling_accuracies.append(accuracy_score(y_test_noisy, pred_mean.argmax(axis=1)))
    disentangling_aleatorics.append(uncertainty(pred_ale_std).mean())
    disentangling_epistemics.append(uncertainty(pred_epi_std).mean())

    entropy_accuracies.append(accuracy_score(y_test_noisy, entropy_preds.mean(axis=0).argmax(axis=1)))
    entropy_aleatorics.append(expected_entropy(entropy_preds).mean())
    entropy_epistemics.append(mutual_information(entropy_preds).mean())
    

In [41]:
plt.plot(noises, disentangling_accuracies, label="Disentangling")
plt.plot(noises, entropy_accuracies, label="Entropy")
plt.legend()
plt.show()

In [42]:
plt.plot(noises, disentangling_aleatorics, label="Aleatoric")
plt.plot(noises, disentangling_epistemics, label="Epistemic")
plt.title("Multi-Head")
plt.xlabel("Noise")
plt.ylabel("Uncertainty")
plt.legend()
plt.show()

In [43]:
plt.plot(noises, entropy_aleatorics, label="Aleatoric")
plt.plot(noises, entropy_epistemics, label="Epistemic")
plt.title("Entropy")
plt.xlabel("Noise")
plt.ylabel("Uncertainty")
plt.legend()
plt.show()